In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import nltk




In [ ]:
train_dataset = pd.read_csv("/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv",encoding="latin")
test_dataset = pd.read_csv("/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv",encoding="latin")

In [ ]:



import collections
from nltk import word_tokenize
queue = collections.deque(maxlen=3) 

vocab = set()
unigram = {}
bigram = {}
trigram = {}

import re
for item in  train_dataset["OriginalTweet"].values :
    
    
    item = item.replace(';','.')
    item = item.replace('?','.')
    item = item.replace('!','.')
    sentences = item.lower().split(".")
    #print(sentences)
    #sentences = re.split('. |?|!', item.lower())
    #print(sentences)
    for sentence in sentences:
        sentence = sentence.replace('@', '')
        sentence = sentence.replace('#', '')
        
        sentence = sentence.replace(',', '')
        sentence = sentence.replace(':', '')
        sentence = sentence.replace('|', '')
        
        queue.clear()
       
        tokens = word_tokenize(sentence)
        for token in tokens:
            queue.append(token)
            #print(queue)
            if token not in vocab:
                vocab.add(token)
            if token not in unigram:
                unigram[token] = 0
            unigram[token] += 1
            
            if len(queue) >= 2:
                i = tuple(queue)[:2]
                if item not in bigram:
                    bigram[i] = 0
                bigram[i] += 1
            if len(queue) == 3:
                i = tuple(queue)[:3]
                if item not in trigram:
                    trigram[i] = 0
                trigram[i] += 1
totalword = len(unigram)
unigram[''] = totalword
    

In [ ]:
totalword

In [ ]:
#def getProbUni(word):
#   if word not in unigram:
#        return 0
#    return unigram[word] / totalword

def getProBigram(word):
    if word not in bigram:
        return 0
    return bigram[word]/unigram[word[0]]
def getProTrigram(word):
    if word not in trigram:
        return 0
    return trigram[word]/bigram[word[:2]]


def find_next_word(words):
    candidate_list = []
    
    # duyệt tất cả từ trong vocab và tính probability nếu từ sau là nó
    for word in vocab:
        #p1 = getProbUni((word))
        p2 = getProBigram((words[-1], word)) #if len(words) >= 2 else 0 
        p3 = getProTrigram((words[-2], words[-1], word)) if len(words) >= 3 else 0
        #phần tử cuois cùng , phần tử cuối cùng thứ 2
        # nội suy
        p = 0.5*p2 + 0.5*p3 # tỉ lệ có tính ra hoặc lựa chọn
        
        candidate_list.append((word, p))
    
    # sắp  xếp giảm dần
    candidate_list.sort(key=lambda x: x[1], reverse=True)
    return candidate_list[0]



In [ ]:
import random 

sentence = []
SENTENCE_LENGTH = 20
#  tìm từ ngẫu nhiên trong tập đã có 
rand_index = random.randint(0, totalword -1)
first_word = list(vocab)[rand_index]
sentence.append(first_word)
print(first_word)
for i in range(SENTENCE_LENGTH-1):
    #print(i)
    word = find_next_word(sentence)
    if word[1] == 0.0 :
        print("khong xuat hien tron tu dien")
        break
    print(word)
    sentence.append(word[0])
print (" ".join(sentence))

In [ ]:
'''

# lấy từ trong tập test
vocab_test = set()
for item in  test_dataset["OriginalTweet"].values :
    
    
    item = item.replace(';','.')
    item = item.replace('?','.')
    item = item.replace('!','.')
    sentences = item.lower().split(".")
    for sentence in sentences:
        sentence = sentence.replace('@', '')
        sentence = sentence.replace('#', '')
        
        sentence = sentence.replace(',', '')
        sentence = sentence.replace(':', '')
        sentence = sentence.replace('|', '')

        tokens = word_tokenize(sentence)
        for token in tokens:
            if token not in vocab:
                vocab_test.add(token)
totalWord_test = len(vocab_test)
totalWord_test


''' 
